# **The Impact of System Instruction Language on the Response Quality of a Tourism-Focused Large Language Model**

# **Pre-requisites**


In [ ]:
!pip install openai

In [ ]:
!pip install transformers torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124
# !pip install torch torchvision

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [ ]:
!pip install peft trl evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 13.5 MB/s eta 0:00:00


In [ ]:
# Env
from google.colab import userdata

# Data
from google.colab import drive
import json
import copy
import html

# OpenAI
import os
# from openai import OpenAI

# HuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, TrainingArguments
from datasets import load_dataset, Dataset
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from trl import SFTTrainer, SFTConfig


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# OPENAI
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
HF_TOKEN = userdata.get('HF_TOKEN')

# **Dataset**

In [ ]:
train_ds = load_dataset("ggeraldo/transformed_dataset", split='train[:90%]')
test_ds = load_dataset("ggeraldo/transformed_dataset", split='train[90%:]')

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

transformed_google_0_999.json:   0%|          | 0.00/8.58M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def preprocess(dataset, eval=False):
  # Returns the full BL_dataset and the ENG_dataset
  BL_dataset = []
  ENG_dataset = []
  for entry in dataset:
    container = {}
    eng_container = {}
    messages = []
    eng_messages = []
    system = entry['messages'][0]
    user = entry['messages'][1]
    asst = entry['messages'][-1]
    # BILINGUAL SETUP
    if not eval:
      messages.extend([system, user, asst])
    else:
      messages.extend([system, user])
      asst.pop('name')
      asst.pop('tool_call_id')
      asst.pop('tool_calls')
    for msg in messages:
      msg.pop('name')
      msg.pop('tool_call_id')
      msg.pop('tool_calls')
    # ENG SETUP
    eng_system = copy.deepcopy(system)
    eng_system['content'] = "You are an AI assistant specializing in Jeju Island tourism. We help users plan their Jeju trips and provide information on tourist attractions, restaurants, accommodations, and more. You can understand and reply in both Korean and English as a tour guide."
    if not eval:
      eng_messages.extend([eng_system, user, asst])
    else:
      eng_messages.extend([eng_system, user])
    container['messages'] = messages
    eng_container['messages'] = eng_messages
    if eval:
      container['reference'] = asst
      eng_container['reference'] = asst
    BL_dataset.append(container)
    ENG_dataset.append(eng_container)
  return BL_dataset, ENG_dataset

BL_dataset, ENG_dataset = preprocess(train_ds)
train_BL_dataset = Dataset.from_list(BL_dataset)
train_ENG_dataset = Dataset.from_list(ENG_dataset)
# Identical because eval=True creates the same dataset essentially
eval_dataset, eval_dataset = preprocess(test_ds, eval=True)


In [ ]:
train_BL_dataset[0]

{'messages': [{'content': '당신은 제주도 관광 전문 AI 어시스턴트입니다. 사용자의 제주 여행 계획을 도와주고, 관광지, 맛집, 숙소 등에 대한 정보를 제공합니다.',
   'role': 'system'},
  {'content': "I'm planning to visit Masichju in Samdo-il-dong, Jeju-si next Tuesday. Can you check if they're open that day?",
   'role': 'user'},
  {'content': "Unfortunately, Masichju will be closed on Tuesday, January 16th.\n\n**Masichju Operation Information**\n\n- Business Hours: 11:00 - 21:00 every day\n- Closed: Every Tuesday\n- Location: Samdo-il-dong, Jeju-si\n\n**Recommended Visit Schedule**\n\nWe recommend that you adjust your visit plans to avoid Tuesdays. If you reschedule your visit to Monday or Wednesday, you will be able to taste Masichju's signature menu items.\n\n**Recommended Menu**\n\nHere are the must-try menu items when you visit Masichju:\n\n1. Stir-fried octopus on a hot plate (10,000-13,000 won)\n - A harmony of spicy yet chewy octopus and crunchy vegetables\n - You can adjust the spiciness so everyone can enjoy it\n\n2. Octopus Rice 

In [ ]:
train_ENG_dataset[0]

{'messages': [{'content': 'You are an AI assistant specializing in Jeju Island tourism. We help users plan their Jeju trips and provide information on tourist attractions, restaurants, accommodations, and more. You can understand and reply in both Korean and English as a tour guide.',
   'role': 'system'},
  {'content': "I'm planning to visit Masichju in Samdo-il-dong, Jeju-si next Tuesday. Can you check if they're open that day?",
   'role': 'user'},
  {'content': "Unfortunately, Masichju will be closed on Tuesday, January 16th.\n\n**Masichju Operation Information**\n\n- Business Hours: 11:00 - 21:00 every day\n- Closed: Every Tuesday\n- Location: Samdo-il-dong, Jeju-si\n\n**Recommended Visit Schedule**\n\nWe recommend that you adjust your visit plans to avoid Tuesdays. If you reschedule your visit to Monday or Wednesday, you will be able to taste Masichju's signature menu items.\n\n**Recommended Menu**\n\nHere are the must-try menu items when you visit Masichju:\n\n1. Stir-fried octo

In [ ]:
with open('eval_bl_dataset.jsonl', 'w') as outfile:
    for entry in BL_dataset:
        json.dump(entry, outfile)
        outfile.write('\n')
with open('train_eng_dataset.jsonl', 'w') as outfile:
    for entry in ENG_dataset:
        json.dump(entry, outfile)
        outfile.write('\n')

# **Qwen**

In [ ]:
qwen_model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_model_name,
    dtype="auto",
    device_map="auto"
)
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_model_name)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

## **Qwen - BL Dataset**

In [ ]:
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_grad_norm = 1.0

qwen_bl_training_args = SFTConfig(
    output_dir="./qwen-bl-lora", # change when training
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=10,
    learning_rate=1e-4,
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio = warmup_ratio,
    lr_scheduler_type = lr_scheduler_type,
    logging_steps=10,
    max_grad_norm=max_grad_norm,
    # save_steps=500,
    save_total_limit=2,
    report_to="none",
    packing=True,
    optim="adamw_torch"
)

qwen_bl_lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Training LLaMa BLLossoM with BL_Dataset
qwen_bl_trainer = SFTTrainer(
    model=qwen_model,
    processing_class=qwen_tokenizer,
    args=qwen_bl_training_args,
    train_dataset=train_BL_dataset,
    peft_config=qwen_bl_lora_config,
)

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
qwen_bl_trainer.train()
qwen_bl_trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.370400
20,1.707800
30,1.521600
40,1.403200
50,1.340400
60,1.305900
70,1.287900
80,1.233700
90,1.224600
100,1.225200


In [ ]:
qwen_bl_trainer.model.save_pretrained("./drive/MyDrive/hyperscale/models/qwen-bl-lora-adapter")
qwen_bl_trainer.save_model("./drive/MyDrive/hyperscale/models/qwen-bl-lora")

## Qwen - ENG Dataset

In [ ]:
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_grad_norm = 1.0

qwen_eng_training_args = SFTConfig(
    output_dir="./qwen-eng-lora-v2", # change when training
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=10,
    learning_rate=1e-4,
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio = warmup_ratio,
    lr_scheduler_type = lr_scheduler_type,
    logging_steps=10,
    max_grad_norm=max_grad_norm,
    # save_steps=500,
    save_total_limit=2,
    report_to="none",
    packing=True,
    optim="adamw_torch"
)

qwen_eng_lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Training LLaMa BLLossoM with BL_Dataset
qwen_eng_trainer = SFTTrainer(
    model=qwen_model,
    processing_class=qwen_tokenizer,
    args=qwen_eng_training_args,
    train_dataset=train_ENG_dataset,
    peft_config=qwen_eng_lora_config,
)

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
qwen_eng_trainer.train()
qwen_eng_trainer.save_model()
qwen_eng_trainer.model.save_pretrained("./drive/MyDrive/hyperscale/models/qwen-eng-lora-adapter-v2")
qwen_eng_trainer.save_model("./drive/MyDrive/hyperscale/models/qwen-eng-lora-v2")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.430000
20,1.719200
30,1.518900
40,1.403700
50,1.342900
60,1.308400
70,1.290300
80,1.236600
90,1.227700
100,1.228200


In [ ]:
prompt = "Can you tell me more about your life?"
messages = [
    {"role": "user", "content": prompt},
]
print("Tokenizing input...")
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False, # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

Tokenizing input...


In [ ]:
print("Generating from model...")
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
print("Decoding output....")
# parse thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

# thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("thinking content:", thinking_content)
print("content:", content)

Generating from model...


# **LLaMa BLLosoM** (Bilingual Kor-Eng LLM)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from trl import SFTTrainer, SFTConfig

In [ ]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
bllossom_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## **LLaMa BLLosoM** - BL Dataset

In [ ]:
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_grad_norm = 1.0

training_args = SFTConfig(
    output_dir="./eval-bllossom-lora", # change when training
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=10,
    learning_rate=1e-4,
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio = warmup_ratio,
    lr_scheduler_type = lr_scheduler_type,
    logging_steps=10,
    max_grad_norm=max_grad_norm,
    # save_steps=500,
    save_total_limit=2,
    report_to="none",
    packing=True,
    optim="adamw_torch"
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Training LLaMa BLLossoM with BL_Dataset
trainer = SFTTrainer(
    model=bllossom_model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=train_BL_dataset,
    eval_dataset=test_BL_dataset,
    peft_config=lora_config,
)

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()
trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss
10,2.227900
20,1.655500
30,1.481000
40,1.413400
50,1.335400
60,1.284400
70,1.268200
80,1.227300
90,1.194900
100,1.180700


In [ ]:
trainer.push_to_hub("ggeraldo/bllossom-tuned-tourist")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

  ...ossom-lora/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

  ...om-lora/training_args.bin:   2%|1         |   115B / 6.22kB            

CommitInfo(commit_url='https://huggingface.co/ggeraldo/bllossom-lora/commit/51a40756c2d02ee5d0d801def3b1ecf278418451', commit_message='ggeraldo/bllossom-tuned-tourist', commit_description='', oid='51a40756c2d02ee5d0d801def3b1ecf278418451', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ggeraldo/bllossom-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='ggeraldo/bllossom-lora'), pr_revision=None, pr_num=None)

In [ ]:
trainer.model.save_pretrained("./drive/MyDrive/hyperscale/models/bllossom-lora-adapter")
trainer.save_model("./drive/MyDrive/hyperscale/models/bllossom-lora")

In [ ]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

## **LLaMa BLLosoM** - ENG Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_grad_norm = 1.0

eng_training_args = SFTConfig(
    output_dir="./llama-eng-lora-v2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio = warmup_ratio,
    lr_scheduler_type = lr_scheduler_type,
    logging_steps=10,
    max_grad_norm=max_grad_norm,
    # save_steps=500,
    save_total_limit=2,
    report_to="none",
    packing=True,
    optim="adamw_torch"
)

eng_lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Training LLaMa BLLossoM with BL_Dataset
eng_trainer = SFTTrainer(
    model=bllossom_model,
    processing_class=tokenizer,
    args=eng_training_args,
    train_dataset=train_ENG_dataset,
    peft_config=eng_lora_config,
)

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
eng_trainer.train()
eng_trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss
10,2.213100
20,1.671600
30,1.474500
40,1.398600
50,1.316300
60,1.283400
70,1.249400
80,1.213200
90,1.176600
100,1.157800


In [ ]:
eng_trainer.model.save_pretrained("./drive/MyDrive/hyperscale/models/llama-eng-lora-adapter-v2")
eng_trainer.save_model("./drive/MyDrive/hyperscale/models/llama-eng-lora-v2")

In [ ]:
eng_trainer.push_to_hub("ggeraldo/eng-bllossom-tuned-tourist")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...om-lora/training_args.bin: 100%|##########| 6.22kB / 6.22kB            

  ...ossom-lora/tokenizer.json:  97%|#########7| 16.7MB / 17.2MB            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

CommitInfo(commit_url='https://huggingface.co/ggeraldo/bllossom-lora/commit/083240d1c3af274842d33a605a1a08fd80f03d77', commit_message='ggeraldo/eng-bllossom-tuned-tourist', commit_description='', oid='083240d1c3af274842d33a605a1a08fd80f03d77', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ggeraldo/bllossom-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='ggeraldo/bllossom-lora'), pr_revision=None, pr_num=None)